# Zadanie domowe -- interpolacja dwusześcienna

Interpolacja dwusześcienna, to podobnie jak w przypadku interpolacji dwuliniowej, rozszerzenie idei interpolacji jednowymiarowej na dwuwymiarową siatkę.
W trakcie jej obliczania wykorzystywane jest 16 pikseli z otoczenia (dla dwuliniowej 4).
Skutkuje to zwykle lepszymi wynikami - obraz wyjściowy jest bardziej gładki i z mniejszą liczbą artefaktów.
Ceną jest znaczny wzrost złożoności obliczeniowej (zostało to zaobserwowane podczas ćwiczenia).

Interpolacja dana jest wzorem:
\begin{equation}
I(i,j) = \sum_{i=0}^{3} \sum_{j=0}^{3} a_{ij} x^i y^j
\end{equation}

Zadanie sprowadza się zatem do wyznaczenia 16 współczynników $a_{ij}$.
W tym celu wykorzystuje się, oprócz wartość w~puntach $A$ (0,0), $B$ (1 0), $C$ (1,1), $D$ (0,1) (por. rysunek dotyczący interpolacji dwuliniowej), także pochodne cząstkowe $A_x$, $A_y$, $A_{xy}$.
Pozwala to rozwiązać układ 16-tu równań.

Jeśli zgrupujemy parametry $a_{ij}$:
\begin{equation}
a = [ a_{00}~a_{10}~a_{20}~a_{30}~a_{01}~a_{11}~a_{21}~a_{31}~a_{02}~a_{12}~a_{22}~a_{32}~a_{03}~a_{13}~a_{23}~a_{33}]
\end{equation}

i przyjmiemy:
\begin{equation}
x = [A~B~D~C~A_x~B_x~D_x~C_x~A_y~B_y~D_y~C_y~A_{xy}~B_{xy}~D_{xy}~C_{xy}]^T
\end{equation}

To zagadnienie można opisać w postaci równania liniowego:
\begin{equation}
Aa = x
\end{equation}
gdzie macierz $A^{-1}$ dana jest wzorem:

\begin{equation}
A^{-1} =
\begin{bmatrix}
1& 0& 0& 0& 0& 0& 0& 0& 0& 0& 0& 0& 0& 0& 0& 0 \\
0&  0&  0&  0&  1&  0&  0&  0&  0&  0&  0&  0&  0&  0&  0&  0 \\
-3&  3&  0&  0& -2& -1&  0&  0&  0&  0&  0&  0&  0&  0&  0&  0 \\
2& -2&  0&  0&  1&  1&  0&  0&  0&  0&  0&  0&  0&  0&  0&  0 \\
0&  0&  0&  0&  0&  0&  0&  0&  1&  0&  0&  0&  0&  0&  0&  0 \\
0&  0&  0&  0&  0&  0&  0&  0&  0&  0&  0&  0&  1&  0&  0&  0 \\
0&  0&  0&  0&  0&  0&  0&  0& -3&  3&  0&  0& -2& -1&  0&  0 \\
0&  0&  0&  0&  0&  0&  0&  0&  2& -2&  0&  0&  1&  1&  0&  0 \\
-3&  0&  3&  0&  0&  0&  0&  0& -2&  0& -1&  0&  0&  0&  0&  0 \\
0&  0&  0&  0& -3&  0&  3&  0&  0&  0&  0&  0& -2&  0& -1&  0 \\
9& -9& -9&  9&  6&  3& -6& -3&  6& -6&  3& -3&  4&  2&  2&  1 \\
-6&  6&  6& -6& -3& -3&  3&  3& -4&  4& -2&  2& -2& -2& -1& -1 \\
2&  0& -2&  0&  0&  0&  0&  0&  1&  0&  1&  0&  0&  0&  0&  0 \\
0&  0&  0&  0&  2&  0& -2&  0&  0&  0&  0&  0&  1&  0&  1&  0 \\
-6&  6&  6& -6& -4& -2&  4&  2& -3&  3& -3&  3& -2& -1& -2& -1 \\
4& -4& -4&  4&  2&  2& -2& -2&  2& -2&  2& -2&  1&  1&  1&  1 \\
\end{bmatrix}
\end{equation}

Potrzebne w rozważaniach pochodne cząstkowe obliczane są wg. następującego przybliżenia (przykład dla punktu A):
\begin{equation}
A_x = \frac{I(i+1,j) - I(i-1,j)}{2}
\end{equation}
\begin{equation}
A_y = \frac{I(i,j+1) - I(i,j-1)}{2}
\end{equation}
\begin{equation}
A_xy = \frac{I(i+1,j+1) - I(i-1,j) - I(i,j-1) + I(i,j)}{4}
\end{equation}

## Zadanie

Wykorzystując podane informacje zaimplementuj interpolację dwusześcienną.
Uwagi:
- macierz $A^{-1}$ dostępna jest w pliku *a_invert.py*
- trzeba się zastanowić nad potencjalnym wykraczaniem poza zakres obrazka (jak zwykle).

Ponadto dokonaj porównania liczby operacji arytmetycznych i dostępów do pamięci koniecznych przy realizacji obu metod interpolacji: dwuliniowej i dwusześciennej.

In [ ]:
#TODO Do samodzielnej implementacji

import ainvert
import numpy as np
import cv2
import os
from matplotlib import pyplot as plt

In [ ]:
def bicubic_interpolation(image, scale_x, scale_y):
    height, width = image.shape
    new_height = int(height * scale_y)
    new_width = int(width * scale_x)
    new_image = np.zeros((new_height, new_width), dtype=np.uint8)
    
    for y in range(1,new_height-3):
        for x in range(1,new_width-3):
            i = int(y / scale_y)
            j = int(x / scale_x)
            i1 = int(np.floor(i))
            j1 = int(np.floor(j))
            i2 = min(i1 + 1, height - 1)
            j2 = min(j1 + 1, width - 1)

            A = image[i1, j1]
            B = image[i1, j2]
            C = image[i2, j1]
            D = image[i2, j2]

            Ax = (image[i1 + 1, j1] - image[i1 - 1, j1]) / 2
            Ay = (image[i1, j1 + 1] - image[i1, j1 - 1]) / 2
            Axy = (image[i1 + 1, j1 + 1] - image[i1 - 1, j1] - image[i1, j1 - 1] + image[i1, j1]) / 4

            Bx = (image[i1 + 1, j2] - image[i1 - 1, j2]) / 2
            By = (image[i1, j2 + 1] - image[i1, j2 - 1]) / 2
            Bxy = (image[i1 + 1, j2 + 1] - image[i1 - 1, j2] - image[i1, j2 - 1] + image[i1, j2]) / 4

            Dx = (image[i2 + 1, j1] - image[i2 - 1, j1]) / 2
            Dy = (image[i2, j1 + 1] - image[i2, j1 - 1]) / 2
            Dxy = (image[i2 + 1, j1 + 1] - image[i2 - 1, j1] - image[i2, j1 - 1] + image[i2, j1]) / 4

            Cx = (image[i2 + 1, j2] - image[i2 - 1, j2]) / 2
            Cy = (image[i2, j2 + 1] - image[i2, j2 - 1]) / 2
            Cxy = (image[i2 + 1, j2 + 1] - image[i2 - 1, j2] - image[i2, j2 - 1] + image[i2, j2]) / 4

            x = [A, B, D, C, Ax, Bx, Dx, Cx, Ay, By, Dy, Cy, Axy, Bxy, Dxy, Cxy]
            a = np.dot(ainvert.A_invert, x)

            for row in range(4):
                for col in range(4):
                    new_image[i, j] += a[row*4 + col] * (i**row) * (j**col) 
    return new_image

In [ ]:
parrot = cv2.imread('parrot.bmp')
parrot = cv2.cvtColor(parrot, cv2.COLOR_BGR2GRAY)

parrot_2=bicubic_interpolation(parrot, 1.5,1.5)

plt.figure(figsize=(parrot_2.shape[0]/100,parrot_2.shape[1]/100), dpi=200)
plt.imshow(parrot_2, cmap ="gray")
plt.xticks([]), plt.yticks([])
plt.show()
            
